In [1]:
import pickle
import numpy as np
from datasets import load_dataset
import tqdm
import math
import sys

In [2]:
# # GLUE
# dataset = load_dataset("glue", "sst2")
# val_dataset = dataset["validation"]

dataset = load_dataset("stanfordnlp/imdb")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

Using the latest cached version of the dataset since stanfordnlp/imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/dhruv/.cache/huggingface/datasets/stanfordnlp___imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Sat Apr 20 22:44:49 2024).


### CNN Based Classifier

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [410]:
# Define the CNN architecture for 1D vectors
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 32, kernel_size = 3)
#         self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3)
#         self.conv3 = nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3)
#         self.pool = nn.MaxPool1d(kernel_size = 2)
        self.fc1 = nn.Linear(768, 512)  # Adjust input size according to your data
        self.fc2 = nn.Linear(512, 1)      # Output layer with 1 neuron for binary classification

    def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = self.pool(nn.functional.relu(self.conv3(x)))
#         x = x.view(-1, 16 * 94)  # Adjust the size according to the output size of the last convolutional layer
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

In [411]:
# Sample data loader for demonstration
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [412]:
# Get Training Data

# Read pkl file 
with open('imdb-train-768.pkl', 'rb') as f:
    vectors = pickle.load(f)
    
X = np.array(vectors)

labels = []
for item in train_dataset:
    labels.append(item['label'])
    
Y = np.array(labels)

### Determining which Subset 

In [770]:
# Read Instance Scores
with open('instance-scores-IMDB-L2-30-40-30.pkl', 'rb') as f:
    instance_scores = pickle.load(f)

In [771]:
X_eff = []
Y_eff = []

M = len(instance_scores)
for i in range(M):
    if instance_scores[i] < -0.005 or instance_scores[i] > 0.025 or 1:
        X_eff.append(X[i])
        Y_eff.append(Y[i])

In [772]:
N = len(X_eff)
X_train = torch.from_numpy(np.array(X_eff).reshape(N, 1, 768))
Y_train = torch.from_numpy(np.array(Y_eff).reshape(N, 1, 1)).float()
print(str(N) + ' samples selected')

25000 samples selected


In [773]:
c0 = 0
c1 = 0
for i in Y_eff:
    if i == 0:
        c0 += 1
    else:
        c1 += 1
        
print('Class 0: ' + str(c0))
print('Class 1: ' + str(c1))

Class 0: 12500
Class 1: 12500


### Start Training

In [774]:
# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.00005)

# Create data loader
trainset = SampleDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle=True)

# Train the model
for epoch in range(25):  # Adjust number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.0692
[1,   110] loss: 0.6816
[1,   210] loss: 0.6481
[1,   310] loss: 0.5975
[1,   410] loss: 0.5426
[1,   510] loss: 0.4875
[1,   610] loss: 0.4459
[1,   710] loss: 0.4175
[2,    10] loss: 0.0356
[2,   110] loss: 0.3704
[2,   210] loss: 0.3567
[2,   310] loss: 0.3580
[2,   410] loss: 0.3553
[2,   510] loss: 0.3351
[2,   610] loss: 0.3300
[2,   710] loss: 0.3321
[3,    10] loss: 0.0359
[3,   110] loss: 0.3186
[3,   210] loss: 0.3141
[3,   310] loss: 0.3066
[3,   410] loss: 0.2998
[3,   510] loss: 0.3100
[3,   610] loss: 0.3087
[3,   710] loss: 0.3013
[4,    10] loss: 0.0296
[4,   110] loss: 0.2911
[4,   210] loss: 0.2862
[4,   310] loss: 0.2954
[4,   410] loss: 0.2971
[4,   510] loss: 0.2918
[4,   610] loss: 0.2843
[4,   710] loss: 0.2960
[5,    10] loss: 0.0284
[5,   110] loss: 0.2963
[5,   210] loss: 0.2877
[5,   310] loss: 0.2720
[5,   410] loss: 0.2855
[5,   510] loss: 0.2782
[5,   610] loss: 0.2824
[5,   710] loss: 0.2596
[6,    10] loss: 0.0312
[6,   110] loss:

In [775]:
# Save the trained model
torch.save(model.state_dict(), 'models/IMDB/model-full.pth')

In [776]:
# Perform inference with the saved model
loaded_model = CNNClassifier()
loaded_model.load_state_dict(torch.load('models/IMDB/model-full.pth'))
loaded_model.eval()

CNNClassifier(
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)

In [777]:
# Example inference
sample_input = torch.randn(1, 1, 768)  # Example input
output = loaded_model(sample_input)
print("Model output:", output.item())

Model output: 1.1369031410746189e-10


In [778]:
# Lets do testing

# Read pkl file 
with open('imdb-test-768.pkl', 'rb') as f:
    test_vectors = pickle.load(f)


In [779]:
gt = []
preds = []

for i in tqdm.tqdm(range(len(test_vectors))):
    input_vector = torch.tensor(test_vectors[i].reshape(1, 1, 768))
    actual = test_dataset[i]['label']
    output = loaded_model(input_vector) 
    if output < 0.5:
        pred = 0
    else:
        pred = 1
    gt.append(actual)
    preds.append(pred)

100%|██████████| 25000/25000 [00:04<00:00, 5038.51it/s]


In [780]:
# Print the number of misclassifications
misc = 0
for i in range(len(gt)):
    if gt[i] != preds[i]:
        misc += 1

print(misc)

2668


In [781]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(gt, preds)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89     12500
           1       0.89      0.90      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



(768,)